# RestFul API - FastAPI


<img style="display: block; margin: auto;" src="https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png" title="git" width="580" height="50">

[FastAPI](https://fastapi.tiangolo.com/) es un web framework moderno y rápido (de alto rendimiento) para construir APIs con `Python 3.7+` basado en las anotaciones de tipos estándar de `Python`.

Sus características principales son:

- **Rapidez:** Alto rendimiento, a la par con `NodeJS` y `Go` (gracias a Starlette y Pydantic). Uno de los frameworks de Python más rápidos.
- **Rápido de programar:** Incrementa la velocidad de desarrollo entre 200% y 300%.
- **Menos errores:** Reduce los errores humanos (de programador) aproximadamente un 40%.
- **Intuitivo:** Gran soporte en los editores con auto completado en todas partes. Gasta menos tiempo debugging.
- **Fácil:** Está diseñado para ser fácil de usar y aprender. Gastando menos tiempo leyendo documentación.
- **Corto:** Minimiza la duplicación de código. Múltiples funcionalidades con cada declaración de parámetros. Menos errores.
- **Robusto:** Crea código listo para producción con documentación automática interactiva.
- **Basado en estándares:** Basado y totalmente compatible con los estándares abiertos para APIs: OpenAPI (conocido previamente como Swagger) y JSON Schema.

**FastAPI** está ganando popularidad entre los Frameworks de `Python`. Su sintaxis también es similar a la de `Flask`, por lo que es fácil cambiar a ella si ha usado `Flask` antes.

[https://christophergs.com/python/2021/06/16/python-flask-fastapi/](https://christophergs.com/python/2021/06/16/python-flask-fastapi/)

![](https://christophergs.com/assets/images/fastapi_flask_post/benchmarks.jpeg)

## 📘 Actividad en clase: Mi primera app con **FastAPI**

En esta actividad vamos a crear nuestra **primera API con FastAPI** usando `uv`.

Trabajaremos de forma progresiva:
1. **Hello World**
2. **Path Parameter**
3. **Body Parameter con Pydantic**

---

### 🚀 Paso 1: Crear el repositorio y clonarlo en local

En GitHub crea un repo que se llame **`my-first-fastapi-app`**.

Clónalo en tu computadora:

```bash
git clone https://github.com/<tu_usuario>/my-first-fastapi-app.git
cd my-first-fastapi-app
```

---

### 🐍 Paso 2: Crear un ambiente virtual con `uv`

Inicializamos el proyecto con `uv`:

```bash
uv init
```

Esto generará un archivo `pyproject.toml`.

---

### 📦 Paso 3: Instalar FastAPI

Instalamos FastAPI con las dependencias estándar:

```bash
uv add fastapi --extra standard
```

---

### 📝 Paso 4: Crear el archivo `main.py`

Crea un archivo `main.py` en la raíz del repo y copia el siguiente código base:

```python
from fastapi import FastAPI
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float

app = FastAPI()
```

---

## 👣 Creación de endpoints

### 1️⃣ Hello World

Agrega el siguiente endpoint al archivo `main.py`:

```python
@app.get("/")
async def hello_world():
    return "Hello World!"
```

Ejecuta la app con `uv run` (sin activar manualmente el venv):

```bash
uv run fastapi dev main.py
```

👉 Abre en el navegador: [http://127.0.0.1:8000](http://127.0.0.1:8000)

---

### 2️⃣ Path Parameter

Ahora agrega este endpoint a tu `main.py`:

```python
@app.get("/api/v1/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}
```

Prueba en tu navegador:
👉 [http://127.0.0.1:8000/api/v1/items/5](http://127.0.0.1:8000/api/v1/items/5)

---

### 3️⃣ Body Parameter con Pydantic

Finalmente, agrega este endpoint **POST**:

```python
@app.post("/api/v1/items/")
async def create_item(item: Item):
    return (f"El item {item.name} es: "
            f"{item.description}, cuesta {item.price}, "
            f"y tiene un impuesto de {item.tax}")
```

👉 Prueba desde la **documentación interactiva**:
[http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

Envía un JSON como este:
```json
{
  "name": "Laptop",
  "description": "Equipo de trabajo",
  "price": 1500.0,
  "tax": 240.0
}
```

---

💡 **Reflexión final**:
Con esta actividad aprendimos cómo:
- Crear un repo y ambiente virtual con `uv`
- Instalar FastAPI
- Desarrollar endpoints con diferentes tipos de parámetros (query, path, body)
- Ejecutar la app con `uv run`
- Usar la documentación interactiva de FastAPI


## 📂 Usando `.env` para manejar secretos

Antes de proteger endpoints, necesitamos una forma **segura de manejar claves y secretos**. En lugar de escribirlos en el código o exportarlos manualmente cada vez, usamos archivos **`.env`**.

### ¿Qué es un `.env`?
- Un archivo de texto con pares `CLAVE=VALOR`.
- Permite centralizar configuraciones y secretos.
- Puede ignorarse en Git (`.gitignore`) para no exponer datos sensibles.
- Ejemplo: distintas claves en desarrollo, pruebas o producción.

---

### Ejemplo de archivo `.env`

Crea un archivo `.env` en la raíz del proyecto:

```
API_KEY=supersecreto
```

---

### Cómo cargar `.env` en FastAPI

Instalamos `python-dotenv` para leer el archivo automáticamente:

```bash
uv add python-dotenv
```

En tu `main.py`:

```python
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

API_KEY = os.getenv("API_KEY")
```

Ahora, cada vez que ejecutes la app, FastAPI podrá acceder a `API_KEY` desde `.env`.

---

💡 **Buenas prácticas con `.env`**
- Nunca subas `.env` a repositorios públicos.
- Usa un archivo `.env.example` para documentar las variables sin valores reales.
- En producción, considera usar un *secret manager* (AWS, GCP, Azure).


## 🔐 Proteger endpoints con API Key (Header)

Para proteger un endpoint con FastAPI vamos a usar una **API Key enviada a través de un header HTTP**.
Por convención se suele llamar `X-API-Key` o `Authorization`.

Ventajas:
- No aparece en la URL.
- No se expone fácilmente en logs o historiales.
- Es más consistente con estándares modernos de autenticación.

---

### Código (`main.py`)

```python
import os
from fastapi import FastAPI, Security, Depends, HTTPException
from fastapi.security import APIKeyHeader
from pydantic import BaseModel
from dotenv import load_dotenv

# ---------- Cargar variables ----------
load_dotenv()
API_KEY = os.getenv("API_KEY")

# ---------- Modelos ----------
class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float

# ---------- App ----------
app = FastAPI(title="API Key Demo")

# ---------- Seguridad por header ----------
api_key_header = APIKeyHeader(name="X-API-Key", description="API key por header", auto_error=True)

async def get_api_key(api_key: str = Security(api_key_header)) -> str:
    if API_KEY and api_key == API_KEY:
        return api_key
    raise HTTPException(status_code=403, detail="Could not validate credentials")

# ---------- Endpoint protegido ----------
@app.get("/api/v1/secure-data/", tags=["secure"])
async def secure_data(api_key: str = Depends(get_api_key)):
    return {"message": "Secure data access granted."}

# ---------- Endpoints de ejemplo previos ----------
@app.get("/")
async def hello_world():
    return "Hello World!"

@app.get("/api/v1/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}

@app.post("/api/v1/items/")
async def create_item(item: Item):
    return (f"El item {item.name} es: "
            f"{item.description}, cuesta {item.price}, "
            f"y tiene un impuesto de {item.tax}")
```

---

### Ejecutar la app

```bash
uv run fastapi dev main.py
```

---

### Probar el endpoint protegido

1. Abre la documentación interactiva:
   👉 [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

   - Haz clic en el botón **Authorize**.
   - Ingresa la clave en el campo `X-API-Key`.
   - Ahora prueba el endpoint `/api/v1/secure-data/`.

   Si la clave es correcta, la respuesta será:

   ```json
   {
     "message": "Secure data access granted."
   }
   ```

2. Alternativamente, puedes usar **Postman**:
   - Método: **GET**
   - URL: `http://127.0.0.1:8000/api/v1/secure-data/`
   - Header:
     ```
     Key: X-API-Key
     Value: supersecreto
     ```
   - Respuesta esperada:
     ```json
     {
       "message": "Secure data access granted."
     }
     ```

## 📚 Actividad: API de Libros con FastAPI + SQLite + API Key (Header)

En esta actividad crearás una API REST que maneja un CRUD de libros usando:
- **FastAPI** para la API
- **SQLite** como base de datos
- **SQLAlchemy** como ORM
- **API Key por header (`X-API-Key`)** para proteger endpoints
- **Versionado** de rutas con `/api/v1/...`

### 1) Crear el repositorio y clonar

```bash
git clone https://github.com/<tu_usuario>/my-fastapi-sqlite-books.git
cd my-fastapi-sqlite-books
```

### 2) Inicializar el proyecto con `uv` e instalar dependencias

```bash
uv init
uv add fastapi --extra standard
uv add sqlalchemy
uv add python-dotenv
```

### 3) Crear archivo `.env`

En la raíz del proyecto crea el archivo `.env`:

```
API_KEY=supersecreto
```

### 4) Estructura esperada

```
my-fastapi-sqlite-books/
├─ books.py
├─ database.py
└─ models.py
```

### 5) Código de la base de datos — `database.py`

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./books.db"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()

### 6) Modelo ORM — `models.py`

In [ ]:
from sqlalchemy import Column, Integer, String
from database import Base

class Books(Base):
    __tablename__ = "books"

    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    author = Column(String)
    description = Column(String)
    rating = Column(Integer)

### 7) App FastAPI + Seguridad + CRUD — `books.py`

In [ ]:
import os
from fastapi import FastAPI, HTTPException, Depends, Security
from fastapi.security import APIKeyHeader
from pydantic import BaseModel, Field
from sqlalchemy.orm import Session
from dotenv import load_dotenv

import models
from database import engine, SessionLocal

# ----- Cargar variables de entorno (.env) -----
load_dotenv()
API_KEY = os.getenv("API_KEY")

# ----- Iniciar app -----
app = FastAPI(title="Books API", version="1.0.0")

# ----- Crear tablas -----
models.Base.metadata.create_all(bind=engine)

# ----- Dependencia DB -----
def get_db():
    try:
        db = SessionLocal()
        yield db
    finally:
        db.close()

# ----- Seguridad por header -----
api_key_header = APIKeyHeader(name="X-API-Key", description="API key por header", auto_error=True)

async def get_api_key(api_key: str = Security(api_key_header)) -> str:
    if API_KEY and api_key == API_KEY:
        return api_key
    raise HTTPException(status_code=403, detail="Could not validate credentials")

# ----- Esquema Pydantic -----
class Book(BaseModel):
    title: str = Field(min_length=1)
    author: str = Field(min_length=1, max_length=100)
    description: str = Field(min_length=1, max_length=100)
    rating: int = Field(gt=-1, lt=101)

# ----- Endpoints -----
@app.get("/")
def root():
    return {"message": "Books API up. See /docs"}

@app.get("/api/v1/books/", tags=["books"])
def list_books(db: Session = Depends(get_db), api_key: str = Depends(get_api_key)):
    return db.query(models.Books).all()

@app.post("/api/v1/books/", tags=["books"])
def create_book(book: Book, db: Session = Depends(get_db), api_key: str = Depends(get_api_key)):
    book_model = models.Books(
        title=book.title,
        author=book.author,
        description=book.description,
        rating=book.rating
    )
    db.add(book_model)
    db.commit()
    db.refresh(book_model)
    return book_model

@app.put("/api/v1/books/{book_id}", tags=["books"])
def update_book(book_id: int, book: Book, db: Session = Depends(get_db), api_key: str = Depends(get_api_key)):
    book_model = db.query(models.Books).filter(models.Books.id == book_id).first()
    if book_model is None:
        raise HTTPException(status_code=404, detail=f"ID {book_id} : Does not exist")

    book_model.title = book.title
    book_model.author = book.author
    book_model.description = book.description
    book_model.rating = book.rating

    db.add(book_model)
    db.commit()
    db.refresh(book_model)
    return book_model

@app.delete("/api/v1/books/{book_id}", tags=["books"])
def delete_book(book_id: int, db: Session = Depends(get_db), api_key: str = Depends(get_api_key)):
    book_model = db.query(models.Books).filter(models.Books.id == book_id).first()
    if book_model is None:
        raise HTTPException(status_code=404, detail=f"ID {book_id} : Does not exist")

    db.query(models.Books).filter(models.Books.id == book_id).delete()
    db.commit()
    return {"deleted_id": book_id}

### 8) Ejecutar la aplicación

```bash
uv run fastapi dev books.py
```

Abre la documentación: [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)
- Haz clic en **Authorize**
- Ingresa `X-API-Key` = el valor de tu `.env`
- Prueba los endpoints `/api/v1/books/`


### 9) Probar también en Postman (opcional)

- URL base: `http://127.0.0.1:8000/api/v1/books/`
- Header requerido:
  ```
  X-API-Key: supersecreto
  ```
- Body ejemplo (POST/PUT):

```json
{
  "title": "Clean Code",
  "author": "Robert C. Martin",
  "description": "Prácticas de código limpio",
  "rating": 95
}
```

# 🧪 Tarea 3 — API de Usuarios con FastAPI + DB

## Objetivo
Crear una **API REST** con **FastAPI** que gestione usuarios y persista información en una **base de datos**. La API debe exponer endpoints para **crear, actualizar, obtener y eliminar usuarios**, respetando reglas de validación (email único) y buenas prácticas de versionamiento (`/api/v1/...`).

---

## 📌 Entregables
- Repositorio **`Tarea3_PCD`** en GitHub.
- Rama de trabajo **`feat/api`** con un **Pull Request** hacia `main` (hacer merge; **no borrar** la rama `feat/api`).
- Archivo principal **`main.py`** con los endpoints.
- Archivo **`README.md`** con instrucciones para correr el proyecto.
- Gestión de dependencias con **`uv`**.
- **Link del repositorio** subido a Canvas.

---

## 📅 Fechas
- **Fecha de entrega:** Jueves 25 de Septiembre de 2025 a las 19:55
- **QUIZ 3:** Martes 23 de Septiembre de 2025 a la hora de clase
  - **Contenido del quiz:** **API y FastAPI**.

---

## ✅ Requisitos funcionales de la API

Crear una nueva API con **cuatro** endpoints bajo el prefijo `/api/v1`:

1) **Crear usuario** → `POST /api/v1/users/`
   - Recibe un objeto con la siguiente estructura y lo guarda en **una tabla** en la **DB**:
   ```json
   {
     "user_name": "name",
     "user_id": 123,
     "user_email": "email@example.com",
     "age": 30,
     "recommendations": ["a", "b", "c"],
     "ZIP": "44100"
   }
   ```
   - `age` y `ZIP` son **opcionales**.
   - `recommendations` es **list[str]**.
   - **Regla:** si el `user_email` ya existe en la tabla, **debe** responder con un **error** indicando que el email está repetido (p. ej., `409 Conflict` o `400 Bad Request`).

2) **Actualizar usuario por id** → `PUT /api/v1/users/{user_id}`
   - Si el `user_id` **no existe**, responder con **error** indicando que no se encontró.

3) **Obtener usuario por id** → `GET /api/v1/users/{user_id}`
   - Si el `user_id` **no existe**, responder con **error** indicando que no se encontró.

4) **Eliminar usuario por id** → `DELETE /api/v1/users/{user_id}`
   - Si el `user_id` **no existe**, responder con **error** indicando que no se encontró.

> 🔐 **Seguridad (requerida):** Los endpoints deben estar **protegidos** con una **API Key** por **header** (p. ej., `X-API-Key`) y validada contra una variable de entorno cargada desde `.env`.

---

## 🗂️ Estructura sugerida del proyecto

```
Tarea3_PCD/
├─ main.py
├─ database.py          # conexión a la DB (SQLite recomendado) + Session
├─ models.py            # modelo(s) SQLAlchemy (tabla users)
├─ .env                 # API_KEY=...
├─ .env.example         # API_KEY=<TU_VALOR_AQUI>  (sin secretos reales)
└─ README.md
```
---

## 🛠️ Requisitos técnicos mínimos

### Ambiente virtual y dependencias con `uv`
1. Inicializar el proyecto:
   ```bash
   uv init
   ```
2. Agregar dependencias:
   ```bash
   uv add fastapi --extra standard
   uv add sqlalchemy
   uv add pydantic
   uv add python-dotenv
   ```
3. Ejecutar la app:
   ```bash
   uv run fastapi dev main.py
   ```

### `README.md` (mínimo)
- Cómo instalar dependencias / crear entorno con `uv`.
- Cómo definir `.env` (añade **`.env.example`**).
- Cómo correr la app.
- Descripción breve de los endpoints.
- Ejemplos de requests.

---

## 💡 Sugerencias de diseño

- Prefijo de rutas: **`/api/v1`** (ej.: `/api/v1/users/`).
- Modelo SQLAlchemy `User` con `user_email` **único**.
- Manejo de errores:
  - `404 Not Found` para `user_id` inexistente.
  - `409 Conflict` (o `400`) si `user_email` ya existe al crear/actualizar.
- Seguridad por header:
  - `X-API-Key` validado contra `API_KEY` en `.env`.

---

## 📤 Entrega
Sube el **link del repositorio** en el espacio de Canvas correspondiente.

---
